In [2]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import keras
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers.core import Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

from keras.utils import plot_model, to_categorical

In [3]:
df = pd.read_csv("FX_MA_25days_10years.csv")
df.head()

,time,open,high,low,close,25mean,target
0,2009-01-28 07:00:00,88.974,90.772,88.890,90.294,90.40976,0
1,2009-01-29 07:00:00,90.294,90.656,89.470,90.036,90.38120,0
2,2009-01-30 07:00:00,90.036,90.155,89.180,89.932,90.30536,0
3,2009-02-01 07:00:00,89.936,90.065,89.733,90.019,90.22872,0
4,2009-02-02 07:00:00,90.009,90.019,88.802,89.447,90.06980,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3165 entries, 0 to 3164
Data columns (total 7 columns):
time      3165 non-null object
open      3165 non-null float64
high      3165 non-null float64
low       3165 non-null float64
close     3165 non-null float64
25mean    3165 non-null float64
target    3165 non-null int64
dtypes: float64(5), int64(1), object(1)
memory usage: 173.2+ KB


In [5]:
# 7 : 3
df.iloc[2550]

time      2017-07-13 06:00:00
open                  113.154
high                  113.532
low                   112.862
close                  113.28
25mean                 112.05
target                      1
Name: 2550, dtype: object

In [6]:
split_data = '2017-06-15 06:00:00'
train, test = df[df['time'] < split_data], df[df['time'] >= split_data]

In [7]:
train = train.set_index('time')
test = test.set_index('time')

In [8]:
X_train = train.iloc[:,:5]
Y_train = train['target']

X_test = test.iloc[:,:5]
Y_test = test['target']

In [9]:
X_train = X_train.as_matrix()
print(X_train)
print("Data len: ",len(X_train))

X_test = X_test.as_matrix()
print(X_train)
print("Data len: ",len(X_test))

# 正規化
ms = MinMaxScaler()
X_train = ms.fit_transform(X_train)

ms = MinMaxScaler()
X_test = ms.fit_transform(X_test)

[[ 88.974    90.772    88.89     90.294    90.40976]
 [ 90.294    90.656    89.47     90.036    90.3812 ]
 [ 90.036    90.155    89.18     89.932    90.30536]
 ...
 [110.342   110.44    109.632   109.941   111.52976]
 [109.938   110.27    109.815   110.066   111.37212]
 [110.054   110.342   108.815   109.564   111.1836 ]]
Data len:  2530
[[ 88.974    90.772    88.89     90.294    90.40976]
 [ 90.294    90.656    89.47     90.036    90.3812 ]
 [ 90.036    90.155    89.18     89.932    90.30536]
 ...
 [110.342   110.44    109.632   109.941   111.52976]
 [109.938   110.27    109.815   110.066   111.37212]
 [110.054   110.342   108.815   109.564   111.1836 ]]
Data len:  635


/home/tim0114gap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/home/tim0114gap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


In [10]:
c = np.array(Y_train)
print("Y_train Target len: ",len(Y_train))

Y_test = np.array(Y_test)
print("Y_train Target len: ",len(Y_test))

Y_train Target len:  2530
Y_train Target len:  635


In [11]:
maxlen = 25
dataTrain = []
targetTrain = []
for i in range(0, len(X_train) - maxlen):
    dataTrain.append(X_train[i: i + maxlen])
    targetTrain.append(Y_train[i + maxlen])

dataTest = []
targetTest = []
for i in range(0, len(X_test) - maxlen):
    dataTest.append(X_test[i: i + maxlen])
    targetTest.append(Y_test[i + maxlen])

In [12]:
X_train = np.array(dataTrain)
Y_train = np.array(targetTrain)

X_test = np.array(dataTest)
Y_test = np.array(targetTest)
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(2505, 25, 5)
(2505,)
(610, 25, 5)
(610,)


In [13]:
# Y_train = Y_train.reshape(len(Y_train), 1)
# Y_test = Y_test.reshape(len(Y_test), 1)

Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

In [14]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(2505, 25, 5)
(2505, 2)
(610, 25, 5)
(610, 2)


In [15]:
'''
モデル設定
'''
length_of_sequence = X_train.shape[1]  # 入力層 = 5
in_out_neurons = 2                     # 出力層 = 2
n_hidden = 100                         # 隠れ層 = 100

model = Sequential()
model.add(LSTM(n_hidden, batch_input_shape=(None, length_of_sequence, 5), return_sequences=False))
model.add(Dropout(0.25))
model.add(Dense(50))
model.add(Activation("sigmoid"))
model.add(Dense(in_out_neurons))
model.add(Activation("softmax"))





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [16]:
optimizer = Adam(lr = 0.001, beta_1 = 0.9, beta_2 = 0.999)
model.compile(loss="categorical_crossentropy", 
              optimizer=optimizer, metrics=['acc'])

In [17]:
#plot_model(model, to_file='model.png')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100)               42400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
activation_1 (Activation)    (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 102       
_________________________________________________________________
activation_2 (Activation)    (None, 2)                 0         
Total params: 47,552
Trainable params: 47,552
Non-trainable params: 0
_________________________________________________________________


In [18]:
# コールバック指定（https://qiita.com/yukiB/items/f45f0f71bc9739830002）
early_stopping = EarlyStopping(monitor='val_loss', 
                               mode='auto', patience=20)
history = model.fit(X_train, Y_train,
                  batch_size = 16, # 論文で使われるのが1, 32, 128, 256, 512
                  epochs=100,
                  validation_split=0.1,
                  callbacks=[early_stopping])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 2254 samples, validate on 251 samples
Epoch 1/100
2254/2254 [==============================] - 6s 3ms/step - loss: 0.7150 - acc: 0.4871 - val_loss: 0.6923 - val_acc: 0.5179
Epoch 2/100
2254/2254 [==============================] - 5s 2ms/step - loss: 0.6944 - acc: 0.5213 - val_loss: 0.6892 - val_acc: 0.5936
Epoch 3/100
2254/2254 [==============================] - 5s 2ms/step - loss: 0.6885 - acc: 0.5368 - val_loss: 0.6837 - val_acc: 0.6375
Epoch 4/100
2254/2254 [==============================] - 5s 2ms/step - loss: 0.6802 - acc: 0.5776 - val_loss: 0.6773 - val_acc: 0.5378
Epoch 5/100
2254/2254 [==============================] - 5s 2ms/step - loss: 0.6620 - acc: 0.6171 - val_loss: 0.6530 - val_acc: 0.6096
Epoch 6/100
2254/2254 [==============================] - 5s 2ms/step - loss: 0.6305 - acc: 0.6424 - val_loss: 0.6414 - val_acc: 0.6175
Epoch 7/100
2254/2254 [==========================

2254/2254 [==============================] - 9s 4ms/step - loss: 0.2713 - acc: 0.8882 - val_loss: 0.1981 - val_acc: 0.9163
Epoch 59/100
2254/2254 [==============================] - 9s 4ms/step - loss: 0.2718 - acc: 0.8842 - val_loss: 0.2976 - val_acc: 0.8805
Epoch 60/100
2254/2254 [==============================] - 9s 4ms/step - loss: 0.2999 - acc: 0.8687 - val_loss: 0.2769 - val_acc: 0.8685
Epoch 61/100
2254/2254 [==============================] - 9s 4ms/step - loss: 0.2661 - acc: 0.8846 - val_loss: 0.2165 - val_acc: 0.9163
Epoch 62/100
2254/2254 [==============================] - 9s 4ms/step - loss: 0.2640 - acc: 0.8864 - val_loss: 0.2237 - val_acc: 0.9004
Epoch 63/100
2254/2254 [==============================] - 9s 4ms/step - loss: 0.2643 - acc: 0.8913 - val_loss: 0.3570 - val_acc: 0.8486
Epoch 64/100
2254/2254 [==============================] - 9s 4ms/step - loss: 0.2820 - acc: 0.8833 - val_loss: 0.2855 - val_acc: 0.8725
Epoch 65/100
2254/2254 [==============================] - 9s 

In [19]:
predict_classes = model.predict_classes(X_test)
predict_classes

array([1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,

In [20]:
predict_classes.shape, Y_test.shape

((610,), (610, 2))

In [21]:
Y_test = np.array([np.argmax(x) for x in Y_test])

In [22]:
pd.crosstab(Y_test, predict_classes, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0,1,All
True,,,
0,244,52,296
1,45,269,314
All,289,321,610
